In [1]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
import pandas as pd
import plotly.express as px

C:\Users\19378\AppData\Local\Temp\ipykernel_1760\422097414.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\19378\AppData\Local\Temp\ipykernel_1760\422097414.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [2]:
df = pd.read_excel("weather dataset.xlsx", sheet_name="data")
df.head()

,country,location_name,latitude,longitude,temperature_celsius,temperature_fahrenheit,condition_text,wind_mph,pressure_mb,pressure_in,...,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,region,sunrise,sunset
0,Afghanistan,Kabul,34.52,69.18,28.8,83.8,Sunny,7.2,1004,29.64,...,130.2,1.2,0.4,7.9,11.1,1,1,1,05:24:00,18:24:00
1,Albania,Tirana,41.33,19.82,27.0,80.6,Partly cloudy,3.8,1006,29.71,...,104.4,3.6,1.8,28.2,29.6,2,3,2,06:04:00,19:19:00
2,Algeria,Algiers,36.76,3.05,28.0,82.4,Partly cloudy,8.1,1014,29.94,...,16.6,63.1,12.6,6.4,7.9,1,1,3,06:16:00,19:21:00
3,Andorra,Andorra La Vella,42.50,1.52,10.2,50.4,Sunny,6.0,1015,29.97,...,68.0,0.2,0.2,0.5,0.8,1,1,4,07:16:00,20:34:00
4,Angola,Luanda,-8.84,13.23,25.0,77.0,Partly cloudy,2.2,1016,30.00,...,147.3,52.8,26.9,139.6,203.3,4,10,5,06:11:00,18:06:00


## Visualization 1&2

In [8]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Read the dataset
df = pd.read_excel("weather dataset.xlsx", sheet_name="data")
df = df[df['wind_direction'] == 'E']

# Create aggregated values
df_agg = df.groupby(['condition_text', 'wind_direction', 'country'], as_index=False).agg({
    'temperature_fahrenheit': 'mean',
    'wind_mph': 'mean'
})

average_temp_country = df.groupby(["condition_text", "wind_direction", "country"])['temperature_fahrenheit'].mean().reset_index()

# Create a Jupyter Dash object
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Define rows and cols layout
app.layout = html.Div([
    # Dashboard Title
    html.H1("World Weather Analysis Dashboard", style={'text-align': 'center'}),
    
    # Input Controls
    html.Div([
        dbc.Row([
            dbc.Col(html.Div([
                html.Label("Select Weather Condition:"),
                dcc.Dropdown(
                    id='weather-condition-dropdown',
                    options=[{'label': cond, 'value': cond} for cond in df_agg['condition_text'].unique()],
                    multi=False, 
                    value=df_agg['condition_text'].unique()[0]  
                )
            ]), width=12),
        ])
    ]),
    
    # Visualizations
    dbc.Row([
        dbc.Col(html.Div([
            dcc.Graph(id='scatter-plot')
        ]), width=6),
        dbc.Col(html.Div([
            dcc.Graph(id='bar-graph')
        ]), width=6),
    ]),
    
    # Textual Descriptions
    dbc.Row([
        dbc.Col(html.Div([
            html.H4("Scatter Plot Description"),
            html.P("This scatter plot represents the relationship between wind speed and temperature. The points are colored by weather condition and wind direction, and the size of the points represents the wind speed.")
        ]), width=6),
        dbc.Col(html.Div([
            html.H4("Bar Graph Description"),
            html.P("The bar graph displays the average temperature by country and weather condition. The bars are grouped by weather condition, and each bar is colored by country.")
        ]), width=6),
    ]),
])

# Callback function for Scatter Plot and Bar Graph
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-graph', 'figure')],
    [Input('weather-condition-dropdown', 'value')]
)
def update_plots(selected_conditions):
    # Filter the dataset 
    filtered_df = df_agg[(df_agg['condition_text'] == selected_conditions)]
    
    # Scatter Plot
    scatter_fig = px.scatter(filtered_df, x='wind_mph', y='temperature_fahrenheit', color='wind_direction', size='wind_mph',
                             hover_data=['country'], title='Temperature vs. Wind Speed')

    # Bar Graph
    bar_fig = px.bar(filtered_df, x='condition_text', y='temperature_fahrenheit', color='country',
                     title='Avg Temperature by Country and Weather condition', barmode='group')

    return scatter_fig, bar_fig

# Run the application
app.run_server(mode='inline', port=8969)


## Visualization 3

In [28]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output


df = pd.read_excel("weather dataset.xlsx", sheet_name="data")
df = df[df['wind_direction'] == 'E']

# Create a Jupyter Dash app
app = JupyterDash(__name__)

# Set up the layout
app.layout = html.Div([
    dcc.RangeSlider(
        id='humidity-slider',
        min=df['humidity'].min(),
        max=df['humidity'].max(),
        step=10,  # Adjust the step to 10
        marks={i: str(i) for i in range(int(df['humidity'].min()), int(df['humidity'].max())+1, 10)},  # Adjust the marks
        value=[df['humidity'].min(), df['humidity'].max()],
        tooltip={'placement': 'bottom'},
        className='mb-5'
    ),
    dcc.Dropdown(
        id='condition-dropdown',
        options=[{'label': condition, 'value': condition} for condition in df['condition_text'].unique()],
        value=df['condition_text'].unique()[0],
        multi=False,
        className='mb-5'
    ),
    dcc.Graph(
        id='bar-graph',
        config={'scrollZoom': False}
    )
])

# Define callback to update bar graph based on humidity slider and condition dropdown
@app.callback(
    Output('bar-graph', 'figure'),
    [Input('humidity-slider', 'value'),
     Input('condition-dropdown', 'value')]
)
def update_bar_graph(selected_humidity_range, selected_condition):
    # Filter the DataFrame based on the selected humidity range and condition
    filtered_df = df[
        (df['humidity'] >= selected_humidity_range[0]) & (df['humidity'] <= selected_humidity_range[1]) &
        (df['condition_text'] == selected_condition)
    ]

    # Create the bar graph
    bar_fig = px.bar(filtered_df, x='country', y='temperature_fahrenheit', color='wind_direction',
                     hover_data=['humidity'],
                     )

    return bar_fig

# Run the app
app.run_server(mode='inline', port=5690)


## Visualization 4

In [51]:
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import plotly.express as px
from jupyter_dash import JupyterDash
import pandas as pd

# Read data
df = pd.read_excel("weather dataset.xlsx", sheet_name="data")

# Create a list of unique weather conditions
weather_conditions = df['condition_text'].unique()

app = JupyterDash(__name__)

app.layout = html.Div([
    html.H3("Weather Conditions:"),
    dcc.RadioItems(
        id='radiobt_conditions', 
        options=[{'value': x, 'label': x} for x in weather_conditions],
        value=weather_conditions[0],
        labelStyle={'display': 'inline-block'}
    ),
    dcc.Graph(id="choropleth"),
])

@app.callback(
    Output("choropleth", "figure"), 
    [Input("radiobt_conditions", "value")])
def display_choropleth(selected_condition):
    
    # Create a filtered dataset
    df_filtered = df[df['condition_text'] == selected_condition]
    
    # Create aggregated values for the average of Temperature
    df_agg = df_filtered.groupby(['country'], as_index=False).agg({'temperature_celsius': 'mean'})
    
    # Check if 'country' is present in the aggregated dataset
    if 'country' not in df_agg.columns:
        raise ValueError("Column 'country' not found in the aggregated dataset.")
    
    # Create the choropleth map using the average of Temperature
    fig = px.choropleth(df_agg, locations='country', locationmode='country names', color='temperature_celsius')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    
    return fig

app.run_server(mode='inline', port=4696)


## Visualization 5

In [20]:
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
from dash import dcc, html
from dash.dependencies import Input, Output

# Read the dataset
df = pd.read_excel("weather dataset.xlsx", sheet_name="data")

app = JupyterDash(__name__)

# Get unique wind directions
wind_directions = df['wind_direction'].unique()

app.layout = html.Div([
    html.H1("Weather Data Visualization"),

    # Radio button for selecting the wind direction
    dcc.RadioItems(
        id='wind-direction-selector',
        options=[{'label': direction, 'value': direction} for direction in wind_directions],
        value=wind_directions[0],  
        labelStyle={'display': 'block'}
    ),

    # Dropdown for selecting the country
    dcc.Dropdown(
        id='country-dropdown',
        style={'width': '50%'}
    ),

    dcc.Graph(id='weather-visualization')
])

# Callback to update the country dropdown options based on the selected wind direction
@app.callback(
    Output('country-dropdown', 'options'),
    Output('country-dropdown', 'value'),
    Input('wind-direction-selector', 'value')
)
def update_country_options(selected_wind_direction):
    # Filter the dataset based on the selected wind direction
    wind_direction_data = df[df['wind_direction'] == selected_wind_direction]

    # Get unique countries for the selected wind direction
    wind_direction_countries = wind_direction_data['country'].unique()

    # Set default value to the first country in the list
    default_value = wind_direction_countries[0]

    return [{'label': country, 'value': country} for country in wind_direction_countries], default_value

# Callback to update the visualization based on the selected wind direction and country
@app.callback(
    Output('weather-visualization', 'figure'),
    Input('wind-direction-selector', 'value'),
    Input('country-dropdown', 'value')
)
def update_weather_visualization(selected_wind_direction, selected_country):
    # Filter the dataset based on the selected wind direction and country
    filtered_data = df[(df['wind_direction'] == selected_wind_direction) & (df['country'] == selected_country)]

    fig = px.bar(
        filtered_data,
        x='wind_mph',
        y='temperature_fahrenheit',
        labels={'wind_mph': 'Wind Speed', 'temperature_fahrenheit': 'Temperature (°F)'},
        title=f'Temperature/Wind speed in {selected_country}',
        color='condition_text',
        color_discrete_sequence=px.colors.qualitative.Set1,
    )

    fig.update_layout(xaxis_title='Wind Speed', yaxis_title='Temperature (°F)')

    return fig

if __name__ == '__main__':
    app.run_server(mode='inline', port=5690)


In [79]:
from jupyter_dash import JupyterDash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd

# Load the datasets
df = pd.read_excel('weather dataset.xlsx', sheet_name='data')
df_agg = df.groupby(['condition_text', 'wind_direction', 'country'], as_index=False).agg({
    'temperature_fahrenheit': 'mean',
    'wind_mph': 'mean'
})

# Initialize the JupyterDash app
app = JupyterDash(__name__)

# Dashboard layout
app.layout = html.Div([
    # Section 1: Weather Analysis Dashboard
    html.H2("Weather Analysis Dashboard", style={'textAlign': 'center', 'backgroundColor': '#ffcc00', 'padding': '10px'}),

    # Dropdown for selecting weather condition
    dcc.Dropdown(
        id='weather-condition-dropdown',
        options=[{'label': cond, 'value': cond} for cond in df_agg['condition_text'].unique()],
        multi=False,
        value=df_agg['condition_text'].unique()[0],
        style={'margin': 'auto', 'width': '50%'}
    ),

    # Scatter Plot and Bar Graph
    html.Div([
        dcc.Graph(id='scatter-plot'),
        dcc.Graph(id='bar-graph')
    ], style={'display': 'flex'}),

    # Descriptions
    html.Div([
        html.Div([
            html.H4(),
            html.P(" The dashboard consists of a scatter plot and a bar graph, and we can interactively explore the relationship between wind speed, temperature, weather conditions, and countries by filtering data with humidity.")
        ], style={'width': '100%'}),
    ], style={'display': 'flex'}),
    
    # Section 2: Humidity and Condition Analysis
    html.Div([
        html.H2("Humidity and Condition Analysis", style={'textAlign': 'center', 'backgroundColor': '#66ff66', 'padding': '10px'}),

    # Descriptions
    html.Div([
        html.Div([
            html.H4(),
            html.P("The interactive dashboard where we can explore the relationship between humidity, weather condition, and temperature by adjusting a range slider and selecting a specific condition. The bar graph dynamically updates to display the average temperature for each country based on the selected parameters.")
        ], style={'width': '30%'}),
        html.Div([
            dcc.RangeSlider(
        id='humidity-slider',
        min=df['humidity'].min(),
        max=df['humidity'].max(),
        step=10,
        marks={i: str(i) for i in range(int(df['humidity'].min()), int(df['humidity'].max()) + 1, 10)},
        value=[df['humidity'].min(), df['humidity'].max()],
        tooltip={'placement': 'bottom'},
        className='mb-5'
    ),

    # Condition Dropdown
    dcc.Dropdown(
        id='condition-dropdown',
        options=[{'label': condition, 'value': condition} for condition in df['condition_text'].unique()],
        value=df['condition_text'].unique()[0],
        multi=False,
        className='mb-5'
    ),

    # Bar Graph based on Humidity and Condition
    dcc.Graph(id='bar-graph-humidity'),
            ], style={'width': '70%'}),
    ], style={'display': 'flex'}),
    
 ]),
      # Section 3: Choropleth Map
    
     html.Div([
    html.H2("Choropleth Map", style={'textAlign': 'center', 'backgroundColor': '#ffff99', 'padding': '10px'}),

    # Radio button for selecting weather condition
    dcc.RadioItems(
        id='radiobt_conditions',
        options=[{'value': x, 'label': x} for x in df['condition_text'].unique()],
        value=df['condition_text'].unique()[0],
        labelStyle={'display': 'inline-block'}
    ),

    # Choropleth Map
    dcc.Graph(id="choropleth"),
       html.P("The  interactive dashboard where we can explore the average temperature distribution across different countries for a selected weather condition using a choropleth map. The choropleth map updates dynamically based on the user's selection through the radio button.")
    
     ]),
    # Section 4: Wind Direction and Country Analysis
     html.Div([
        html.Div([
             html.Div([
    html.H2("Wind Direction and Country Analysis", style={'textAlign': 'center', 'backgroundColor': '#cc99ff', 'padding': '10px'}),

    # Wind Direction Selector
    dcc.RadioItems(
        id='wind-direction-selector',
        options=[{'label': direction, 'value': direction} for direction in df['wind_direction'].unique()],
        value=df['wind_direction'].unique()[0],
        labelStyle={'display': 'block'}
    ),

    # Country Dropdown
    dcc.Dropdown(
        id='country-dropdown',
        style={'width': '50%'}
    ),

    # Weather Visualization based on Wind Direction and Country
    dcc.Graph(id='weather-visualization')
        ])
            ], style={'width': '70%'}),
        html.Div([
            html.P('The interactive dashboard for exploring weather data, allowing users to select a specific wind direction and country to visualize the relationship between wind speed, temperature, and weather conditions through a dynamically updating bar chart using chained call back functions.')
            ], style={'width': '30%'}),
    ], style={'display': 'flex'}),   
])

# Callback function for Scatter Plot and Bar Graph
@app.callback(
    [Output('scatter-plot', 'figure'),
     Output('bar-graph', 'figure')],
    [Input('weather-condition-dropdown', 'value')]
)
def update_plots(selected_conditions):
    # Filter the dataset
    filtered_df = df_agg[(df_agg['condition_text'] == selected_conditions)]

    # Scatter Plot
    scatter_fig = px.scatter(filtered_df, x='wind_mph', y='temperature_fahrenheit', color='wind_direction',
                             size='wind_mph',
                             hover_data=['country'], title='Temperature vs. Wind Speed')

    # Bar Graph
    bar_fig = px.bar(filtered_df, x='condition_text', y='temperature_fahrenheit', color='country',
                     title='Avg Temperature by Country and Weather condition', barmode='group')

    return scatter_fig, bar_fig

# Callback to update bar graph based on humidity slider and condition dropdown
@app.callback(
    Output('bar-graph-humidity', 'figure'),
    [Input('humidity-slider', 'value'),
     Input('condition-dropdown', 'value')]
)
def update_bar_graph(selected_humidity_range, selected_condition):
    # Filter the DataFrame based on the selected humidity range and condition
    filtered_df = df[
        (df['humidity'] >= selected_humidity_range[0]) & (df['humidity'] <= selected_humidity_range[1]) &
        (df['condition_text'] == selected_condition)
    ]

    # Create the bar graph
    bar_fig = px.bar(filtered_df, x='country', y='temperature_fahrenheit', color='wind_direction',
                     hover_data=['humidity'],
                     )

    return bar_fig

# Callback to update choropleth map based on selected weather condition
@app.callback(
    Output("choropleth", "figure"),
    [Input("radiobt_conditions", "value")]
)
def display_choropleth(selected_condition):
    # Create a filtered dataset
    df_filtered = df[df['condition_text'] == selected_condition]

    # Create aggregated values for the average of Temperature
    df_agg = df_filtered.groupby(['country'], as_index=False).agg({'temperature_celsius': 'mean'})

    # Check if 'country' is present in the aggregated dataset
    if 'country' not in df_agg.columns:
        raise ValueError("Column 'country' not found in the aggregated dataset.")

    # Create the choropleth map using the average of Temperature
    fig = px.choropleth(df_agg, locations='country', locationmode='country names', color='temperature_celsius')
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})

    return fig

# Callback to update the country dropdown options based on the selected wind direction
@app.callback(
    [Output('country-dropdown', 'options'),
     Output('country-dropdown', 'value')],
    Input('wind-direction-selector', 'value')
)
def update_country_options(selected_wind_direction):
    # Filter the dataset based on the selected wind direction
    wind_direction_data = df[df['wind_direction'] == selected_wind_direction]

    # Get unique countries for the selected wind direction
    wind_direction_countries = wind_direction_data['country'].unique()

    # Set default value to the first country in the list
    default_value = wind_direction_countries[0]

    return [{'label': country, 'value': country} for country in wind_direction_countries], default_value

# Callback to update the visualization based on the selected wind direction and country
@app.callback(
    Output('weather-visualization', 'figure'),
    [Input('wind-direction-selector', 'value'),
     Input('country-dropdown', 'value')]
)
def update_weather_visualization(selected_wind_direction, selected_country):
    # Filter the dataset based on the selected wind direction and country
    filtered_data = df[(df['wind_direction'] == selected_wind_direction) & (df['country'] == selected_country)]

    fig = px.bar(
        filtered_data,
        x='wind_mph',
        y='temperature_fahrenheit',
        labels={'wind_mph': 'Wind Speed', 'temperature_fahrenheit': 'Temperature (°F)'},
        title=f'Temperature/Wind speed in {selected_country}',
        color='condition_text',
        color_discrete_sequence=px.colors.qualitative.Set1,
    )

    fig.update_layout(xaxis_title='Wind Speed', yaxis_title='Temperature (°F)')

    return fig

# Run the app
if __name__ == '__main__':
    app.run_server(mode='external', port=6647)


Dash app running on http://127.0.0.1:6647/
